In [7]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [8]:
#set the rows and columns length
BOARD_ROWS = 5
BOARD_COLS = 5

In [9]:
START = (1, 2)
WIN_STATE = (3, 1)
HOLE_STATE = [(-1,-1),(-1,2),(-1,3),(0,1),(0,2),(0,3),(1,1),(1,2),(2,-1),(2,1),(2,2),(3,3)]

In [10]:
#class state defines the board and decides reward, end and next position
class State:
    def __init__(self, state=START):
        #initalise the state to start and end to false
        self.state = state
        self.isEnd = False        

    def getReward(self):
        #give the rewards for each state -5 for loss, +1 for win, -1 for others
        for i in HOLE_STATE:
            if self.state == i:
                return -5
        if self.state == WIN_STATE:
            return 1       
        
        else:
            return -1

    def isEndFunc(self):
        #set state to end if win/loss
        if (self.state == WIN_STATE):
            self.isEnd = True
            
        for i in HOLE_STATE:
            if self.state == i:
                self.isEnd = True

    def nxtPosition(self, action):     
        #set the positions from current action - up, down, left, right
        if action == 0:                
            nxtState = (self.state[0] - 1, self.state[1]) #up             
        elif action == 1:
            nxtState = (self.state[0] + 1, self.state[1]) #down
        elif action == 2:
            nxtState = (self.state[0], self.state[1] - 1) #left
        else:
            nxtState = (self.state[0], self.state[1] + 1) #right


        #check if next state is possible
        if (nxtState[0] >= 0) and (nxtState[0] <= 4):
            if (nxtState[1] >= 0) and (nxtState[1] <= 4):    
                    #if possible change to next state                
                    return nxtState 
        #Return current state if outside grid     
        return self.state 


In [11]:
class Agent:

    def __init__(self):
        #inialise states and actions 
        self.states = []
        self.actions = [1,3,3]    # up=0, down=1, left=2, right=3
        self.State = State()
        #set the learning and greedy values
        self.alpha = 0.5
        self.gamma = 0.8
        self.epsilon = 0.1
        self.isEnd = self.State.isEnd

        # array to retain reward values for plot
        self.plot_reward = []
        
        #initalise Q values as a dictionary for current and new
        self.Q = {}
        self.new_Q = {}
        #initalise rewards to 0
        self.rewards = 0
        
        #initalise all Q values across the board to 0, print these values
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                for k in range(len(self.actions)):
                    self.Q[(i, j, k)] =0
                    self.new_Q[(i, j, k)] = 0
        
        print(self.Q)
        
    

 
    def Action(self):
        #random value vs epsilon
        rnd = random.random()
        #set arbitraty low value to compare with Q values to find max
        mx_nxt_reward =-10
        action = None
        
        #9/10 find max Q value over actions 
        if(rnd >self.epsilon) :
            #iterate through actions, find Q  value and choose best 
            for k in self.actions:
                
                i,j = self.State.state
                
                nxt_reward = self.Q[(i,j, k)]
                
                if nxt_reward >= mx_nxt_reward:
                    action = k
                    mx_nxt_reward = nxt_reward
                    
        #else choose random action
        else:
            action = np.random.choice(self.actions)
        
        #select the next state based on action chosen
        position = self.State.nxtPosition(action)
        return position,action
    
    
    #Q-learning Algorithm
    def Q_Learning(self,episodes):
        x = 0
        #iterate through best path for each episode
        while(x < episodes):
            #check if state is end
            if self.isEnd:
                #get current rewrard and add to array for plot
                reward = self.State.getReward()
                self.rewards += reward
                self.plot_reward.append(self.rewards)
                
                #get state, assign reward to each Q_value in state
                i,j = self.State.state
                for a in self.actions:
                    self.new_Q[(i,j,a)] = round(reward,3)
                    
                #reset state
                self.State = State()
                self.isEnd = self.State.isEnd
                
                #set rewards to zero and iterate to next episode
                self.rewards = 0
                x+=1
            else:
                #set to arbitrary low value to compare net state actions
                mx_nxt_value = -10
                #get current state, next state, action and current reward
                next_state, action = self.Action()
                i,j = self.State.state
                reward = self.State.getReward()
                #add reward to rewards for plot
                self.rewards +=reward
                
                #iterate through actions to find max Q value for action based on next state action
                for a in self.actions:
                    nxtStateAction = (next_state[0], next_state[1], a)
                    q_value = (1-self.alpha)*self.Q[(i,j,action)] + self.alpha*(reward + self.gamma*self.Q[nxtStateAction])
                
                    #find largest Q value
                    if q_value >= mx_nxt_value:
                        mx_nxt_value = q_value
                
                #next state is now current state, check if end state
                self.State = State(state=next_state)
                self.State.isEndFunc()
                self.isEnd = self.State.isEnd
                
                #update Q values with max Q value for next state
                self.new_Q[(i,j,action)] = round(mx_nxt_value,3)
            
            #copy new Q values to Q table
            self.Q = self.new_Q.copy()
        #print final Q table output
        print(self.Q)
        
    #plot the reward vs episodes
    def plot(self,episodes):
        
        plt.plot(self.plot_reward)
        plt.show()
        
        
    #iterate through the board and find largest Q value in each, print output
    def showValues(self):
        for i in range(0, BOARD_ROWS):
            print('-----------------------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                mx_nxt_value = -10
                for a in self.actions:
                    nxt_value = self.Q[(i,j,a)]
                    if nxt_value >= mx_nxt_value:
                        mx_nxt_value = nxt_value
                out += str(mx_nxt_value).ljust(6) + ' | '
            print(out)
        print('-----------------------------------------------')

In [12]:
        
if __name__ == "__main__":
    #create agent for 10,000 episdoes implementing a Q-learning algorithm plot and show values.
    ag = Agent()
    episodes = 10000
    ag.Q_Learning(episodes)
    ag.plot(episodes)
    ag.showValues()

{(0, 0, 0): 0, (0, 0, 1): 0, (0, 0, 2): 0, (0, 1, 0): 0, (0, 1, 1): 0, (0, 1, 2): 0, (0, 2, 0): 0, (0, 2, 1): 0, (0, 2, 2): 0, (0, 3, 0): 0, (0, 3, 1): 0, (0, 3, 2): 0, (0, 4, 0): 0, (0, 4, 1): 0, (0, 4, 2): 0, (1, 0, 0): 0, (1, 0, 1): 0, (1, 0, 2): 0, (1, 1, 0): 0, (1, 1, 1): 0, (1, 1, 2): 0, (1, 2, 0): 0, (1, 2, 1): 0, (1, 2, 2): 0, (1, 3, 0): 0, (1, 3, 1): 0, (1, 3, 2): 0, (1, 4, 0): 0, (1, 4, 1): 0, (1, 4, 2): 0, (2, 0, 0): 0, (2, 0, 1): 0, (2, 0, 2): 0, (2, 1, 0): 0, (2, 1, 1): 0, (2, 1, 2): 0, (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0, (2, 3, 0): 0, (2, 3, 1): 0, (2, 3, 2): 0, (2, 4, 0): 0, (2, 4, 1): 0, (2, 4, 2): 0, (3, 0, 0): 0, (3, 0, 1): 0, (3, 0, 2): 0, (3, 1, 0): 0, (3, 1, 1): 0, (3, 1, 2): 0, (3, 2, 0): 0, (3, 2, 1): 0, (3, 2, 2): 0, (3, 3, 0): 0, (3, 3, 1): 0, (3, 3, 2): 0, (3, 4, 0): 0, (3, 4, 1): 0, (3, 4, 2): 0, (4, 0, 0): 0, (4, 0, 1): 0, (4, 0, 2): 0, (4, 1, 0): 0, (4, 1, 1): 0, (4, 1, 2): 0, (4, 2, 0): 0, (4, 2, 1): 0, (4, 2, 2): 0, (4, 3, 0): 0, (4, 3, 1): 0, (4, 3

KeyError: (1, 2, 3)